In [1]:
# Neural Network for binary classification and regression tasks

In [2]:
import numpy as np

In [3]:
# with sigmoid as activation (you can remove sigmoid to perform regression tasks)
# if you want to perform multi-label classification fuck you
class NeuralNetwork:
    def __init__(self, no_of_layers):
        self.num_layers = no_of_layers
        num_neurons = []
        hidden_count = 1
        for i in range(no_of_layers):
            if i != no_of_layers - 1:
                print("Enter Number of Neurons for Hidden Layer "+str(hidden_count))
                num_neurons.append(int(input()))
                hidden_count += 1
            else:
                print("Number of neurons in output layer = 1")
                num_neurons.append(1)
                
            
        self.numb_neurons = num_neurons
        self.weights = []
        self.bias = []
        self.a = []
        self.dz = []
        self.delta_w = []
        self.delta_b = []
        self.y_pred = None
        self.trained = False
        self.lr = 0
        self.m = 0
    def train(self, train_x, train_y, epochs, lr):
        # weight and bias initializations
        if self.trained == False:
            for i in range(self.num_layers):
                if i == 0:
                   self.weights.append(np.random.randn(self.numb_neurons[i],train_x.shape[1]))
                else:
                   self.weights.append(np.random.randn(self.numb_neurons[i],self.numb_neurons[i - 1]))
                self.bias.append(np.random.randn(self.numb_neurons[i],1))
                self.y_pred = np.zeros((1,len(train_y)))
                self.lr = lr
                self.m = len(train_x)
        self.trained = True
        for i in range(epochs):    
            self.forward_pass(train_x)
            self.backward_pass(train_x, train_y)
            y_t = train_y.reshape(1, len(train_y))
            print(self.loss_func(y_t, self.y_pred))

    def test(self, x_test):
        a_ = []
        
        for i in range(self.num_layers):
            if i == 0:
                z = np.matmul(self.weights[i], x_test.T) + self.bias[i]
                a = 1/(1+np.exp(-z))
                a_.append(a)
            else:
                z = np.matmul(self.weights[i], a_[i-1]) + self.bias[i]
                a = 1/(1+np.exp(-z))
                a_.append(a)
            y_p = a[-1].flatten()
            # only for classification tasks
            for i in range(len(y_p)):
                if y_p[i] >= 0.5:
                   y_p[i] = 1
                else:
                   y_p[i] = 0
        return y_p

    
    def loss_func(self, y_true, y_pred):
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)) 
    
    def forward_pass(self, train_x):
        for i in range(self.num_layers):
            if i == 0:
                z = np.matmul(self.weights[i], train_x.T) + self.bias[i]
                a = 1/(1+np.exp(-z)) # remove this to perform regression tasks
                self.a.append(a)
            else:
                z = np.matmul(self.weights[i], self.a[i-1]) + self.bias[i]
                a = 1/(1+np.exp(-z))
                self.a.append(a)

    def backward_pass(self, train_x, train_y):
        for i in range(self.num_layers - 1, -1,-1):
            # last layer case where derivative of loss is needed
            if i == self.num_layers - 1:
                dz = self.a[i] - train_y
                self.dz.insert(0, dz)
                delta_w = 1/self.m * np.matmul(dz ,self.a[i-1].T)  
                self.delta_w.insert(0, delta_w)
                self.delta_b.insert(0, 1/self.m * np.sum(dz))
            # first layer case where a[i-1] is actually train_x
            elif i == 0:               
                dz = np.matmul(self.weights[i+1].T, self.dz[0]) * (self.a[i] * (1 - self.a[i]))
                self.dz.insert(0, dz)
                delta_w = 1/self.m * np.matmul(dz, train_x)
                self.delta_w.insert(0, delta_w)
                self.delta_b.insert(0, 1/self.m * np.sum(dz))
            # all hidden layer cases
            else:
                dz = np.matmul(self.weights[i+1].T, self.dz[0]) * (self.a[i] * (1 - self.a[i]))
                self.dz.insert(0, dz)
                delta_w = 1/self.m * np.matmul(dz, self.a[i-1].T)
                self.delta_w.insert(0, delta_w)
                self.delta_b.insert(0, 1/self.m * np.sum(dz))
        # updating weights
        for i in range(len(self.weights)):
            self.weights[i] -= self.lr * self.delta_w[i]
            self.bias[i] -= self.lr * np.broadcast_to(self.delta_b[i],self.bias[i].shape)
        self.y_pred = self.a[-1]
        self.dz, self.a, self.deltas  = [],[],[]                

In [4]:
nn = NeuralNetwork(6)

Enter Number of Neurons for Hidden Layer 1


 100


Enter Number of Neurons for Hidden Layer 2


 32


Enter Number of Neurons for Hidden Layer 3


...

Enter Number of Neurons for Hidden Layer 4


 8


Enter Number of Neurons for Hidden Layer 5


 4


Number of neurons in output layer = 1


In [5]:
# clean and normalize your data here
train_x = np.array([[0,0],[0,1],[1,0],[1,1]])
train_y = np.array([0,0,0,1])

In [6]:
# train_x should be scaled and of shape (training examples, features) i.e (batch_size, input)
# train_y should be a one dimensional vector
# both train_x and train_y must be numpy arrays
# 3rd param is no. of epochs
# 4th param is learning rate
nn.train(train_x,train_y,10000, 0.1)

0.7574224042488662
0.7402647016311756
0.7240087146086284
0.7086167254087277
0.6940454264660995
0.6802491561893798
0.6671833933646862
0.6548083670583491
0.6430924801605813
0.6320150285715146
0.6215674932595924
0.6117526375898151
0.602580945135504
0.5940646685407974
0.5862107265786023
0.5790143746602429
0.5724554724541382
0.566498224349745
0.5610939984595604
0.5561859363992435
0.5517138918324955
0.547618628578234
0.543844789092482
0.5403426139769265
0.537068648640751
0.5339857472592691
0.5310626539278125
0.5282733715940328
0.5255964587207086
0.5230143371644449
0.5205126552454951
0.5180797248400864
0.5157060366169952
0.5133838497210752
0.5111068486188886
0.5088698586600158
0.5066686120270913
0.504499556462064
0.5023597000782993
0.500246486495656
0.49815769537621074
0.49609136416127436
0.49404572741729086
0.49201917070510737
0.490010196314012
0.4880173985677435
0.48603944672954436
0.4840750738181573
0.4821230699033963
0.48018227868250074
0.47825159634871767
0.4763299719521721
0.47441640862

In [7]:
nn.test(np.array([[0,0],[0,1],[1,0],[1,1]]))

array([0., 0., 0., 1.])